In [ ]:
import io
import datetime
import pandas as pd
from PIL import Image
import requests
import io
import glob, os, sys, time, uuid

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw

from video_indexer import VideoIndexer
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.face.models import TrainingStatusType
from msrest.authentication import CognitiveServicesCredentials

In [ ]:
KEY = "4fe196a0811c42cbbd581c45e87a65d7"
ENDPOINT = "https://face-rtw-e2.cognitiveservices.azure.com/"

face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

face_client.api_version

In [ ]:
CONFIG = {
    'SUBSCRIPTION_KEY': '9842b344d288408e821ccd0bb3663003',
    'LOCATION': 'trial',
    'ACCOUNT_ID': '8ed8e2db-9516-4b89-ab8c-29662c57f6c8'
}

video_analysis = VideoIndexer(
    vi_subscription_key=CONFIG['SUBSCRIPTION_KEY'],
    vi_location=CONFIG['LOCATION'],
    vi_account_id=CONFIG['ACCOUNT_ID']
)

In [ ]:
video_analysis.check_access_token()

In [ ]:
video_id = '9e9c7becca'

In [ ]:
video_analysis.get_video_info(video_id)

In [ ]:
info = video_analysis.get_video_info(video_id, video_language='English')

In [ ]:
if len(info['videos'][0]['insights']['faces'][0]['thumbnails']):
    print("We found {} faces in this video.".format(str(len(info['videos'][0]['insights']['faces'][0]['thumbnails']))))

In [ ]:
info['videos'][0]['insights']['faces'][0]['thumbnails']

In [ ]:
images = []
img_raw = []
img_strs = []
for each_thumb in info['videos'][0]['insights']['faces'][0]['thumbnails']:
    if 'fileName' in each_thumb and 'id' in each_thumb:
        file_name = each_thumb['fileName']
        thumb_id = each_thumb['id']
        img_code = video_analysis.get_thumbnail_from_video_indexer(video_id,  thumb_id)
        img_strs.append(img_code)
        img_stream = io.BytesIO(img_code)
        img_raw.append(img_stream)
        img = Image.open(img_stream)
        images.append(img)

In [ ]:
for img in images:
    print(img.info)
    plt.figure()
    plt.imshow(img)

In [ ]:
i = 1
for img in images:
    print(type(img))
    img.save('../material_preparation_step/digital_id_template/faces/human-face' + str(i) + '.jpg')
    i= i+ 1

In [ ]:
keyframes = []
for shot in info["videos"][0]["insights"]["shots"]:
    for keyframe in shot["keyFrames"]:
        keyframes.append(keyframe["instances"][0]['thumbnailId'])

In [ ]:
for keyframe in keyframes:
    img_str = video_analysis.get_thumbnail_from_video_indexer(video_id,  keyframe)

In [ ]:
info['summarizedInsights']['sentiments']

In [ ]:
info['summarizedInsights']['emotions']

# Face

In [ ]:
PERSON_GROUP_ID = str(uuid.uuid4())
person_group_name = 'person-avkash'

In [ ]:
def build_person_group(client, person_group_id, pgp_name):
    print('Create and build a person group...')
    # Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
    print('Person group ID:', person_group_id)
    client.person_group.create(person_group_id = person_group_id, name=pgp_name)

    # Create a person group person.
    human_person = client.person_group_person.create(person_group_id, pgp_name)
    # Find all jpeg human images in working directory.
    human_face_images = [file for file in glob.glob('../material_preparation_step/digital_id_template/faces/*') if file.startswith("human-face")]

    # Add images to a Person object
    for image_p in human_face_images:
        with open(image_p, 'rb') as w:
            client.person_group_person.add_face_from_stream(person_group_id, human_person.person_id, w)

    # Train the person group, after a Person object with many images were added to it.
    client.person_group.train(person_group_id)

    # Wait for training to finish.
    while (True):
        training_status = client.person_group.get_training_status(person_group_id)
        print("Training status: {}.".format(training_status.status))
        if (training_status.status is TrainingStatusType.succeeded):
            break
        elif (training_status.status is TrainingStatusType.failed):
            client.person_group.delete(person_group_id=PERSON_GROUP_ID)
            sys.exit('Training the person group has failed.')
        time.sleep(5)

In [ ]:
try:
    build_person_group(face_client, PERSON_GROUP_ID, person_group_name)
except Exception as e:
    print(e.response.content)

In [ ]:
'''
Detect all faces in query image list, then add their face IDs to a new list.
'''
def detect_faces(client, query_images_list):
    print('Detecting faces in query images list...')

    face_ids = {} # Keep track of the image ID and the related image in a dictionary
    for image_name in query_images_list:
        image = open(image_name, 'rb') # BufferedReader
        print("Opening image: ", image.name)
        time.sleep(5)

        # Detect the faces in the query images list one at a time, returns list[DetectedFace]
        faces = client.face.detect_with_stream(image)  

        # Add all detected face IDs to a list
        for face in faces:
            print('Face ID', face.face_id, 'found in image', os.path.splitext(image.name)[0]+'.jpg')
            # Add the ID to a dictionary with image name as a key.
            # This assumes there is only one face per image (since you can't have duplicate keys)
            face_ids[image.name] = face.face_id

    return face_ids

In [ ]:
test_images = [file for file in glob.glob('/Users/rwhitcom/Documents/ai_engineer_nanodegree/automated_passenger_boarding_kiosk/material_preparation_step/digital_id_template/faces/*')]
test_images

In [ ]:
ids = detect_faces(face_client, test_images)
ids

In [ ]:
ids.keys()

In [ ]:
# Verification example for faces of the same person.
verify_result = face_client.face.verify_face_to_face(ids['/Users/rwhitcom/Documents/ai_engineer_nanodegree/automated_passenger_boarding_kiosk/material_preparation_step/digital_id_template/faces/avkash.png'], ids['/Users/rwhitcom/Documents/ai_engineer_nanodegree/automated_passenger_boarding_kiosk/material_preparation_step/digital_id_template/faces/extracted_avkash.png'])
if verify_result.is_identical:
    print("Faces are of the same (Positive) person, similarity confidence: {}.".format(verify_result.confidence))
else:
    print("Faces are of different (Negative) persons, similarity confidence: {}.".format(verify_result.confidence))

In [ ]:
def show_image_in_cell(face_url):
    response = requests.get(face_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(10,5))
    plt.imshow(img)
    plt.show()

In [ ]:
dl_source_url = 'https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/ca-dl-sample.png'

In [ ]:
show_image_in_cell(dl_source_url)